In [1]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import sparse
from scipy.stats.mstats import gmean
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss
from datetime import datetime
from sklearn.model_selection import train_test_split,StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score
seed = 1234


# Load Data

In [27]:
data_path = "../input/"
train_X = pd.read_pickle(data_path + 'train_X_0319.pkl')
test_X = pd.read_pickle(data_path + 'test_X_0319.pkl')
train_y = np.ravel(pd.read_pickle(data_path + 'train_y_0319.pkl'))
sub_id = test_X.listing_id.astype('int32').values
# all_features = features_to_use + desc_sparse_cols + feat_sparse_cols
print train_X.shape, test_X.shape, train_y.shape

In [29]:
train_X_2.columns.values

array(['listing_id', 'bathrooms', 'bedrooms', 'building_id',
       'display_address', 'latitude', 'longitude', 'manager_id', 'price',
       'street_address', 'feature_1_month_free', 'feature_24/7_concierge',
       'feature_24/7_doorman', 'feature_24/7_doorman_concierge',
       'feature_actual_apt._photos', 'feature_air_conditioning',
       'feature_all_pets_ok', 'feature_all_utilities_included',
       'feature_assigned-parking-space', 'feature_attended_lobby',
       'feature_backyard', 'feature_balcony', 'feature_basement_storage',
       'feature_basketball_court', 'feature_bike_room',
       'feature_bike_storage', 'feature_billiards_room',
       'feature_billiards_table_and_wet_bar', 'feature_brand_new',
       'feature_breakfast_bar', 'feature_bright', 'feature_brownstone',
       'feature_building-common-outdoor-space', 'feature_business_center',
       'feature_cable/satellite_tv', 'feature_cable_ready',
       'feature_call/text_abraham_caro_@_917-373-0862',
       'feat

In [25]:
def RFC_cv(max_features = 0.5, n_estimators=300, min_samples_leaf  =1):
    scores=[]
    est=RandomForestClassifier(max_features=max_features,
                               n_estimators=int(n_estimators),
                               min_samples_leaf =int(min_samples_leaf), 
                               criterion = 'entropy',
                               random_state=seed,
                               n_jobs = 6
                              )
#     est.fit(X_train, y_train)
#     y_val_pred = est.predict_proba(X_val)
    return cross_val_score(est,train_X,train_y, scoring = 'neg_log_loss', cv = 3).mean()
#     return -1*log_loss(y_val, y_val_pred)

In [26]:
cv_score = -1
for x in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:#
    score = RFC_cv(max_features = x)
    if score > cv_score:
        max_features = x
        cv_score = score
    print 'max_features: {0:.1f}\tscore: {1:.12f}'.format(x,score)

max_features: 0.1	score: -0.582229439116
max_features: 0.2	score: -0.580688823867


KeyboardInterrupt: 

In [7]:
print max_features
# -0.576001690038

0.2


In [8]:
for x in [1,2,4,8,16,32]:
    score = RFC_cv(max_features = max_features,min_samples_leaf = x)
    if score > cv_score:
        min_samples_leaf = x
        cv_score = score    
    print 'min_samples_leaf: {0:.1f}\tscore: {1:.12f}'.format(x,score)

min_samples_leaf: 1.0	score: -0.576001690038
min_samples_leaf: 2.0	score: -0.572683987384
min_samples_leaf: 4.0	score: -0.572554897002
min_samples_leaf: 8.0	score: -0.573315740814
min_samples_leaf: 16.0	score: -0.579171901865
min_samples_leaf: 32.0	score: -0.585210249999


In [9]:
for x in [3,5,6,7]:
    score = RFC_cv(max_features = max_features,min_samples_leaf = x)
    if score > cv_score:
        min_samples_leaf = x
        cv_score = score    
    print 'min_samples_leaf: {0:.1f}\tscore: {1:.12f}'.format(x,score)

min_samples_leaf: 3.0	score: -0.572017902570
min_samples_leaf: 5.0	score: -0.572010307343
min_samples_leaf: 6.0	score: -0.572720035720
min_samples_leaf: 7.0	score: -0.572281413819


In [10]:
print min_samples_leaf

5


In [11]:
def RFC_blend(est, train_x, train_y, test_x, fold):
    N_params = len(est)
    print ("Blend %d estimators for %d folds" % (N_params, fold))
    skf = KFold(n_splits=fold,random_state=seed)
    N_class = len(set(train_y))
    
    train_blend_x = np.zeros((train_x.shape[0], N_class*N_params))
    test_blend_x_mean = np.zeros((test_x.shape[0], N_class*N_params))
    test_blend_x_gmean = np.zeros((test_x.shape[0], N_class*N_params))
    scores = np.zeros((fold,N_params))
    best_rounds = np.zeros((fold, N_params))    
    
    for j, ester in enumerate(est):
        print ("Model %d:" %(j+1))
        test_blend_x_j = np.zeros((test_x.shape[0], N_class*fold))

            
        for i, (train_index, val_index) in enumerate(skf.split(train_x)):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x.iloc[train_index]
            train_y_fold = train_y[train_index]
            val_x_fold = train_x.iloc[val_index]
            val_y_fold = train_y[val_index]            
            

            ester.fit(train_x_fold,train_y_fold)
            
            val_y_predict_fold = ester.predict_proba(val_x_fold)
            score = log_loss(val_y_fold, val_y_predict_fold)
            print ("Score: ", score)
            scores[i,j]=score            
            
            train_blend_x[val_index, (j*N_class):(j+1)*N_class] = val_y_predict_fold
            test_blend_x_j[:,(i*N_class):(i+1)*N_class] = ester.predict_proba(test_x)
            
            print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            

        test_blend_x_mean[:,(j*N_class):(j+1)*N_class] = \
                np.stack([test_blend_x_j[:,range(0,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(1,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(2,N_class*fold,N_class)].mean(1)]).T
        
        test_blend_x_gmean[:,(j*N_class):(j+1)*N_class] = \
                np.stack([gmean(test_blend_x_j[:,range(0,N_class*fold,N_class)], axis=1),
                          gmean(test_blend_x_j[:,range(1,N_class*fold,N_class)], axis=1),
                          gmean(test_blend_x_j[:,range(2,N_class*fold,N_class)], axis=1)]).T
            
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x_mean, test_blend_x_gmean, scores,best_rounds)

In [18]:
est = [RandomForestClassifier(max_features=max_features,
                              n_estimators=1000,
                              min_samples_leaf = min_samples_leaf,
                              random_state=seed,
                              criterion = 'entropy',
                              n_jobs = -1
                             )]

(train_blend_x_RFC,
 test_blend_x_RFC_mean,
 test_blend_x_RFC_gmean,
 blend_scores_RFC,
 best_rounds_RFC) = RFC_blend(est,
                              train_X,train_y,
                              test_X,
                              5)

Blend 1 estimators for 10 folds
Model 1:
Model 1 fold 1
('Score: ', 0.5827002039626572)
Model 1 fold 1 fitting finished in 586.761s
Model 1 fold 2


KeyboardInterrupt: 

In [13]:
test_blend_x_RFC_mean

array([[ 0.49637808,  0.42607755,  0.07754437],
       [ 0.94515239,  0.04412316,  0.01072445],
       [ 0.77769542,  0.19479507,  0.02750951],
       ..., 
       [ 0.52137987,  0.36487019,  0.11374994],
       [ 0.40766447,  0.40763976,  0.18469578],
       [ 0.82629405,  0.1489874 ,  0.02471855]])

In [14]:
test_blend_x_RFC_gmean

array([[ 0.495858  ,  0.42554749,  0.07735382],
       [ 0.94513655,  0.04384463,  0.01057997],
       [ 0.77746233,  0.19391488,  0.02702627],
       ..., 
       [ 0.5211564 ,  0.36462256,  0.11357881],
       [ 0.40742575,  0.4075852 ,  0.18421182],
       [ 0.82624148,  0.14871735,  0.02448697]])

In [15]:
# now = datetime.now()

name_train_blend = '../blend/train_blend_RFC_entropy_data0319_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../blend/test_blend_RFC_entropy_mean_data0319_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_gmean = '../blend/test_blend_RFC_entropy_gmean_data0319_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'


print (np.mean(blend_scores_RFC,axis=0))
# print (np.mean(best_rounds_RFC,axis=0))
np.savetxt(name_train_blend,train_blend_x_RFC, delimiter=",")
np.savetxt(name_test_blend_mean,test_blend_x_RFC_mean, delimiter=",")
np.savetxt(name_test_blend_gmean,test_blend_x_RFC_gmean, delimiter=",")

[ 0.56446636]


In [17]:
sub_name = '../output/sub_RFC_entropy_gmean_data0319_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

out_df = pd.DataFrame(test_blend_x_RFC_gmean[:,:3])
out_df.columns = ["low", "medium", "high"]
out_df["listing_id"] = sub_id
out_df.to_csv(sub_name, index=False)

In [20]:
def RFC_cv(max_features = 0.5, n_estimators=300, min_samples_leaf  =1):
    scores=[]
    est=RandomForestClassifier(max_features=max_features,
                               n_estimators=int(n_estimators),
                               min_samples_leaf =int(min_samples_leaf), 
                               criterion = 'gini',
                               random_state=seed,
                               n_jobs = 6
                              )
#     est.fit(X_train, y_train)
#     y_val_pred = est.predict_proba(X_val)
    return cross_val_score(est,train_X,train_y, scoring = 'neg_log_loss', cv = 3).mean()

In [21]:
cv_score = -1
for x in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,'sqrt','log2']:
    score = RFC_cv(max_features = x)
    if score > cv_score:
        max_features = x
        cv_score = score
    print x,'\t', score

0.1 	-0.569883938063
0.2 	-0.568070201511
0.3 	-0.580255504105
0.4 	-0.580217149284
0.5 	-0.578342462823
0.6 	-0.587835546682
0.7 	-0.587667956574
0.8 	-0.586837887517
0.9 	-0.591146951995
sqrt 	-0.571892652393
log2 	-0.581505306336


In [22]:
for x in [0.03,0.06]:
    score = RFC_cv(max_features = x)
    if score > cv_score:
        max_features = x
        cv_score = score
    print x,'\t', score

0.03 	-0.584458999291
0.06 	-0.572849586525


In [23]:
print max_features

0.2


In [24]:
for x in [1,2,4,8,16,32]:
    score = RFC_cv(max_features = max_features,min_samples_leaf = x)
    if score > cv_score:
        min_samples_leaf = x
        cv_score = score    
    print x, '\t', score

1 	-0.568070201511
2 	-0.565360951091
4 	-0.565146344834
8 	-0.568557271357
16 	-0.5734925495
32 	-0.580673259122


In [25]:
print min_samples_leaf

4


In [26]:
est = [RandomForestClassifier(max_features=max_features,
                              n_estimators=1000,
                              min_samples_leaf = min_samples_leaf,
                              random_state=seed,
                              criterion = 'gini',
                              n_jobs = -1
                             )]

(train_blend_x_RFC,
 test_blend_x_RFC_mean,
 test_blend_x_RFC_gmean,
 blend_scores_RFC,
 best_rounds_RFC) = RFC_blend(est,
                              train_X,train_y,
                              test_X,
                              10)

Blend 1 estimators for 10 folds
Model 1:
Model 1 fold 1
('Score: ', 0.5513698824945944)
Model 1 fold 1 fitting finished in 144.779s
Model 1 fold 2
('Score: ', 0.53997090835910089)
Model 1 fold 2 fitting finished in 149.119s
Model 1 fold 3
('Score: ', 0.56048575820075353)
Model 1 fold 3 fitting finished in 148.220s
Model 1 fold 4
('Score: ', 0.53926515109170547)
Model 1 fold 4 fitting finished in 149.259s
Model 1 fold 5
('Score: ', 0.56563756057585568)
Model 1 fold 5 fitting finished in 147.023s
Model 1 fold 6
('Score: ', 0.55302355871567566)
Model 1 fold 6 fitting finished in 148.402s
Model 1 fold 7
('Score: ', 0.56744936754818032)
Model 1 fold 7 fitting finished in 148.894s
Model 1 fold 8
('Score: ', 0.58282378042480076)
Model 1 fold 8 fitting finished in 148.924s
Model 1 fold 9
('Score: ', 0.57826009521767685)
Model 1 fold 9 fitting finished in 144.165s
Model 1 fold 10
('Score: ', 0.56823594580496339)
Model 1 fold 10 fitting finished in 145.219s
Score for model 1 is 0.560652
Score fo

In [42]:
now = datetime.now()

name_train_blend = '../blend/train_blend_RFC_gini_data0319_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../blend/test_blend_RFC_gini_mean_data0319_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_gmean = '../blend/test_blend_RFC_gini_gmean_data0319_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'


print (np.mean(blend_scores_RFC,axis=0))
# print (np.mean(best_rounds_RFC,axis=0))
np.savetxt(name_train_blend,train_blend_x_RFC, delimiter=",")
np.savetxt(name_test_blend_mean,test_blend_x_RFC_mean, delimiter=",")
np.savetxt(name_test_blend_gmean,test_blend_x_RFC_gmean, delimiter=",")

[ 0.56023491]
